In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/clusters/cluster6/T6-1-1K.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/T6-1-1K')

In [ ]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN, device=gpu, floatX=float32"


# import necessary modules
import tensorflow
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
np.random.seed(2017) 
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
import cv2
import pickle
import shutil

In [ ]:
#-----------CLASSIFICATION--------------

# load pre-trained model
model = VGG19(weights='imagenet', include_top=True)
# display model layers
model.summary()

In [ ]:
dir = "/content/T6-1-1K"
time_start = timer()

for movie in os.listdir(dir):
    mov_timer = timer()
    movie_predictions = []
    movie_dir = dir + "/" + movie
    for frame in os.listdir(dir + "/" + movie):
        img = image.load_img(os.path.join(movie_dir,frame), target_size=(224, 224))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
    
        # predict the output 
        preds = model.predict(img)
        movie_predictions.append(decode_predictions(preds, top=3)[0])
        
    pickle.dump(movie_predictions, (open(f"/content/output/labels/{movie}.p", "wb")))

print(f"TOTAL exectution time: {timer() - time_start} seconds.")

In [ ]:
dir = '/content/output/labels'
drive = '/content/drive/My Drive/output/labels'

for movie in os.listdir(dir):
  if(movie.endswith('.p')):
    shutil.copy2(f'{dir}/{movie}', f'{drive}/{movie}')